# Homework 4: Final Report Part 1
This homework is very different from previous homeworks. The most signficant change is that this is a __group assignment__ that focuses on your project. You should complete it in collaboration with members of your project group. There is no coding, so you should feel free to complete the assignment in word or whatever format you are most comfortable with. This will be worth a significant amount of your final grade, so please try to do a good job. 

As usual, the homework is broken into tasks, however, each task is a phase of planning. For each task, you're highly encouraged to include graphics that aid in your explanations and thought processes. If you need help making these graphics, I recommend taking a look at [yed](https://www.yworks.com/products/yed), a really neat graph editor.

Only one member of the group should submit this assignment, however, please provide the names of each member of the group here:

**Group Members:**
*   James Gan
*   Jason Cui
*   Shine Lin (no contribution to ML yet)



## Task 1: Problem Description
For this task you'll be describing what you want to accomplish. Of course some of this can be copy pasted from previous presentations you've done, but there are additional considerations you'll have to make. Specifically you should:
* Describe the goal of your project

    Table tennis is a popular Olympic sport with very few professional players in most countries. It is expensive and can be difficult to learn with low availability of ways to improve. As a result, our overall goal is to build an electronic device that enables an efficient learning experience for table tennis. The device should be capable of detecting data from a collection of sensors, processing that data with standard computation and pre-trained models for real-time analysis, and to send that data to the cloud for more complex machine learning analysis,

* What requirements does the project have? For example, what accuracy do you need to create a compelling experience? How fast does it need to run? Are there any privacy concerns? Etc...

    Our project does not have extreme accuracy requirements: we are targeting beginners, so anything that has a high degree of accuracy towards suggesting the right direction (aka. binary classification) of change for improvement will be sufficient for our design. As a result, we **do want** a high accuracy in the direction of our outputs (e.g. was that game good or bad) but **do not need** high accuracy for more precise purposes (e.g. change the vertical acceleration of the swing 1m/s in order to vastly improve).

    For any pre-trained model we want to put onto our device's on-board Raspberry Pi, we will need it to run very fast: we want it to display results in near-real-time such that players would be able to seamlessly analyze their plays. However, for any models that are not fast, we would put them on the cloud for **post match analysis** which would arguably be fine even if they had up to an hour of delay. **We do not have privacy concerns**: all the data collected is not particularly sensitive - we do not have any biometric or otherwise personally identifiable information.

* What class of machine learning problem is this? Is there overlap between several classes? In this context, a class of problem would be simple classification, natural language processing, vision, or one of the other topics discussed.

    Our project requires a mix of machine learning and data mining algorithms to fulfill the following tasks:

    * Classify whether a player's swing is good or bad by looking at the data of swing movement for the period before the paddle hits the ball,

    **Classification, Clustering**

    * Predict who the current player is by looking at the data of swing movement,

    **Classification, Clustering**

    * Predict quality scores of hits by looking at the data of swing movement for the period before the paddle hits the ball.

    **Regression**

## Task 2: Dataset Description
In this task, you'll consider the dataset being collected to achieve your goal.
* Which features do you need to collect? How do each of these features relate to the output of your network?

    Raw input features: IMU acceleration/IMU angular velocity ($F^6$), piezoelectric disk reading/vibration ($N^3$)

    Manually labelled features: Player Quality ($B$), Player Id ($N$)

    Engineering features: Ball Hitting Event ($N$), Ball Hitting Temporal Dispersity (BHTD)/Ball Hitting Spatial Dispersity (BHSD)/Ball Hitting Strength (BHS) ($F^3\times N$), Top-K Signal In Frequency Domain ($F\times N$)

    Engineering features are features that are generated from raw input through certain functions. Ultimately, a mix of any kind of features are *directly* fed into the network for training. The network output contains Player Quality, Player Id, BHTD/BHSD/BHS only.


* Describe your data collection procedure. How will you store the data? How will you prevent accidentally teaching the network something you didn't mean to. For example, If I trained a network with images all from the same room, it might learn to look for that background and not work anywhere else.

    Given the real-time stream nature of sensor data and batch-processed near-real-time nature of enginnering features, the data is stored in a circular buffer residing in main memory first, then "dumped"/serialized to JSON files (when a threshold of # of pieces of data is reached) and saved both locally and to the cloud (Firebase Cloud Storage).

    To prevent overfitting during data collection, we will ask testers to "swing the paddle and serve the ball multiple times", which will create a rich dataset that differs in the frequency of swings, the direction/angle of swings (which slightly affects gyroscope), and quality of hits. Since we plan to only collect a relatively small amount of data, we do not have to select testers based on ping-pong players' statistical distribution.


* How many samples do you think you'll need? This relates both to the difficulty of the problem and the complexity of the network. How long do you expect it'll take to collect those samples? How much memory do you expect your dataset to take?
 
    We will need 10 samples each containing at least 15 "swing & hit" pairs. This gives as 150 "swing & hit" pairs in total, and each pair will be the data collected from sensors during ~30 seconds. Given the current sample rate of 5 data points/sec and 40 Bytes per data point, each pair will take 6KB. The dataset will be at least 9MB in memory and probably 2x that size in persist media.

    We will need to ask 10 different players to play around 1 minute each, which theoretically results to a total of 10 minutes. In reality, this should take a minimum of half an hour to an hour at most, including finding users, set-up, transition, and post-processing time between test users.


* What are the labels for your data? How are they being labeled? Do you think there's any bias that might be accidentally inserted?

    We will be using a Good/Bad player label. We will determine how to label the players in a subjective manner by considering their background (self-reported beginner versus expert) and each cast a vote for how to label that player. Bias may be inserted given our subjective/non-quantitative approach to estimating a player's performance. Additionally, personal relationships (e.g. giving a close friend a "Good Player" label) may also result in bias.

## Task 3: Model Design
Now that we have our goal and dataset set up, let's think about how to build a model that achieves our goal. For this task, you absolutely should include a graphic that shows the structure of your proposed architecture.
* What type of architecture will you use? What are the pros and cons of your choice? Are there any alternative architectures that might work too?

  * Depending on the deployment type of the machine learning task, we need to consider the supported operations and architectures by TF Lite as we design and train the model.

  * <b>TF Version</b>

    Input -> SimpleRNN -> Dense -> BatchNorm -> Dropout -> BatchNorm -> Dropout -> Dense (Output)

  * <b>TF Lite Version</b>

    Input -> Conv1D -> Flatten -> Dense -> BatchNorm -> Dropout -> Dense -> BatchNorm -> Dropout -> Dense (Output)

    TF Lite does not support RNN nor ELU activations, so we replaced the RNN layer with a Conv1D, and the ELU layer with a ReLU instead.

  * <b>Pros & Cons</b>

    SimpleRNN requires significantly fewer parameters and has simpler architecture. This change is good for avoiding overfitting with insufficient training data, but it loses LSTM's support for long-term memory. The loss of long term memory may not have a significant negative impact, however, given that each sequence usually has a length of 11~14 and does not exceed 16 temporal data points.

    Dense layers are used in conjunction with ELU activation to filter out the negative part of the gradient, which boosts training accuracy. Dense layers are also disabled using bias to force the model to learn the gradient instead of bias. This is good for preventing the model from learning a zero weight matrix and constant bias matrix: this was occuring before such limitations were implemented. However, this is bad because it results in greater overfitting; we mitigated this overfitting effect with multiple layers of Dense-Dropouts.

    BatchNorm layers are applied to "amplify" the gradient as it propagates back through multiple layers of Dense-Dropouts. This is good for making sure that the SimpleRNN layers can learn from slight variations in data, but is bad in that it increases overfitting since it increases model parameters.

    In summary, the model is designed intentionally for extremely small channels to rigorously prevent overfitting. The small number of parameters may affect the stability of the loss curve as epochs grow, but the negative effect is generally managable.

  * <b>Alternatives</b>

    We tried the following architecture and its variants. All of them have a similar (but slightly worse) loss or similar stability as epochs increase.

       * LSTM. All RNN/LSTM layers must not include dropout or recurrent_dropout, which severly push all weight matrices to approximately zero. L1/L2 Regularization does not help much to prevent this issue.
       
       * Residual Learning.

        Input -> SimpleRNN (i)
        Input -> Flatten -> Dense (ii)

        (i) + (ii) -> BatchNorm -> Dropout -> Dense -> BatchNorm -> Dropout -> Dense (Output)

        This approach forces the RNN layer to learn the residual (output - identity), which aims to prevent the model from learning zero weights. However, this would result in instability and overfitting.


* Show the structure of your proposed architecture

    <img src="https://i.imgur.com/PnHIH5O.jpg"></img>

* Is there anything special about your architecture, such as a structural difference not covered in class?

  * Disabled learning bias in all Dense layers but not the RNN layer. This makes sure that RNN layer deals with bias in the input (if any), while forcing the model to use weights (instead of bias) to try fitting the labels.

  * Sigmoid used for regresison. Since the task is unsupervised (hit quality scores), we use sigmoid on the output layer.

  * Mean Absolute Error. We do not use mean square error because the task is regression, not classification. Square errors also weights too much over the (smaller) L1 norm of the target. We do not use percentage error as this would, in contrast, discriminate too much over the (larger) L1 norm of the target. Mean absolute error is the middle ground here.

  * Zero-padded input sequence. All sequences 

## Task 4: Training
Let's now consider how you'll be training the model described. Here, consider how difficult the model might be train and what special considerations you may need to take for your project.
* What hyperparameters make sense for your project? Hyperparameters include:
    * Optimization algorithm (sgd, adam, rmsprop, etc)
   
   We weill use adam optimization with careful activation functions (ELU which filtered out the negative part, and is helpful for boosting up the training process) and layer configurations (including disabled bias except RNN layer forces the model to learn non-zero weights), adam will help dynamically tweak the learning rate.
   
    * Training time (epochs)
    
    Training time is set to 1024 for a variety of reasons:
    
      * Loss on training set starts to converge and forms a asymptote before epoch 1024th.
      * Loss on test set becomes stable and no significant overfitting is found empirically around epoch 1024th.
      * Training is very fast per epoch (a few hundred milliseconds on a Nvidia RTX 2080), and 1024 epochs only takes 1-2 minutes.
      
    *  Regularization (weight decay)
      
      We have attempted a few regularization methods (such as L1, L1_L2, and L2) when the first layer (SimpleRNN or LSTM) was configured to have recurrent dropouts. Such trials were in an attempt to force the model to only learn useful weights to prevent overfitting. However, if configured with non-zero recurrent dropout, the model either learns nothing (always predicts a near-constant despite the input) or overfits. The only stable solution was not to use recurrent dropouts. In this case, regularization was optional.
    
    
* Are there any special training considerations you need to make? For example, reinforcement learning has a very different flow from normal training. What are those adjustments?

    See above described in "Regularization".
    
    
* How will you evaluate the progress of your training. Do you think your validation accuracy will reflect real life?

    Since the type of task is regression (aka. predicting a float value between 0-1 per output), evaluation of the training was given by mean absolute error. The reason why we did not use mean squared error or other types of errors are described in previous sections.
    
    The validation accuracy will paritally reflect the physical limitations of the data, however, for every dataset we instructed each ping pong player a lot to try to standardize data. In real life, however, no assumption could be made of the potential users, which would require more architecture and data collection.

## Task 5: Deployment
Finally, you have a trained model that accomplishes your initial goals. Now, consider how you'll take this model and make something useful from it.
* Where will the model be run? Does it make sense to have it running on an Azure server? Why or why not? Does it make sense to have it running locally (on raspberry pi for example)? Why or why not?

The model will be run locally on a Raspberry Pi. It makes sense to do so because we want to be able to have persistent on-board data storage, and the ability to upload that data onto the cloud for usage in a web-based analytics dashboard. We considered having less complex hardware on our device, but those two features alone would be hard to accomplish without requiring, at least, connection to a phone from an Arduino. Additionally, by running it on a local Raspberry Pi, we are able to do our processing in near real-time for immediate display on an e-ink screen on the device.

Our data which is transmitted to the cloud is then able to be rendered by our analytics dashboard: at this point, we have the ability to add more complex machine learning on a cloud server for display on the analytics dashboard. However, we are not certain that we will actually do this, as we believe a pre-trained model could accomplish the same result without suffering significant speed or battery issues.

By running our model locally, we are able to reduce the cost and overhead of paid cloud services. However, it does mean that the battery life of our device will suffer, and the cost may be higher than with an alternative design. Ultimately, this will reduce the speed of processing, but allow us to have near real-time results, and we are hoping that the speed and battery issues do not interfere with our MVP and demo: a custom battery solution and optimized code would help mitigate these problems during a real product release.

* How many users do you think could use your product at the same time? Will it effect performance?

The way that we have designed our product is currently infinitely scalable with no effect on performance: all computation is performed locally on Raspberry Pis, and the analytics dashboard is created using Javascript, meaning that data visualization processing is done on the user's machine. If we used more complex models on the cloud, then there would be a large cost to maintaining the product performance and usage at scale.

* How will you test how well your final product works?

There are no significant scalability tests required. As a result, we will test how well our final product works through regular usage testing for the 3 days leading up to our final milestone deadline for HW/SW Lab 1.